In [175]:
#carregando bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go 
import plotly.express as px

In [121]:
#Funções
def create_metadata(df):
    df_aux = pd.DataFrame(data={
    'colunas':df.columns,
    'missing':df.isna().sum(),
    'qtd':df.shape[0]
    })
    df_aux['tx_miss'] = round(df_aux['missing']/df_aux['qtd'],2)
    return df_aux

In [122]:
#carregando base de dados de mercado
df_market = pd.read_csv('../data/estaticos_market.csv')

In [123]:
#tamanho do dataframe
df_market.shape

(462298, 182)

## Análise Exploratória dos dados

### 1. Criação dataframe com metadados

In [124]:
#calcular missing values 
df_aux = pd.DataFrame(data={
    'colunas':df_market.columns,
    'tipos':df_market.dtypes,
    'missing':df_market.isna().sum(),
    'qtd':df_market.shape[0],
    'unicos':df_market.nunique()
})

In [125]:
df_aux

,colunas,tipos,missing,qtd,unicos
Unnamed: 0,Unnamed: 0,int64,0,462298,462298
id,id,object,0,462298,462298
fl_matriz,fl_matriz,bool,0,462298,2
de_natureza_juridica,de_natureza_juridica,object,0,462298,67
sg_uf,sg_uf,object,0,462298,6
...,...,...,...,...,...
qt_funcionarios_24meses,qt_funcionarios_24meses,float64,358724,462298,762
tx_crescimento_12meses,tx_crescimento_12meses,float64,388410,462298,2237
tx_crescimento_24meses,tx_crescimento_24meses,float64,388284,462298,3388
tx_rotatividade,tx_rotatividade,float64,358724,462298,2548


In [126]:
#calcular taxa de missing values
df_aux['tx_miss'] = round(df_aux['missing']/df_aux['qtd'],2)

In [5]:
#exportando metadados para CSV
df_aux.to_csv('metadados.csv',index=False)

### 2. Exploração dos dados faltantes

In [127]:
#calculando % das variáveis acima de 15% de dados faltantes
tx_missing = np.arange(0.1,1.00,0.01)
total_missing = list()
for tx in tx_missing:
    total_missing = df_aux.query(f'tx_miss >= {tx}').shape[0]/df_aux.shape[0]
    print(f"Total de campos com dados >= {round(tx*100,2)}% faltantes- {df_aux.query(f'tx_miss >= {tx}').shape[0]}")
    print(f"Porcentagem de variáveis com >= {round(tx*100,2)}% de dados faltantes: {round(total_missing*100,2)}%")
    print('\n')

Total de campos com dados >= 10.0% faltantes- 144
Porcentagem de variáveis com >= 10.0% de dados faltantes: 79.12%


Total de campos com dados >= 11.0% faltantes- 143
Porcentagem de variáveis com >= 11.0% de dados faltantes: 78.57%


Total de campos com dados >= 12.0% faltantes- 143
Porcentagem de variáveis com >= 12.0% de dados faltantes: 78.57%


Total de campos com dados >= 13.0% faltantes- 143
Porcentagem de variáveis com >= 13.0% de dados faltantes: 78.57%


Total de campos com dados >= 14.0% faltantes- 141
Porcentagem de variáveis com >= 14.0% de dados faltantes: 77.47%


Total de campos com dados >= 15.0% faltantes- 141
Porcentagem de variáveis com >= 15.0% de dados faltantes: 77.47%


Total de campos com dados >= 16.0% faltantes- 141
Porcentagem de variáveis com >= 16.0% de dados faltantes: 77.47%


Total de campos com dados >= 17.0% faltantes- 141
Porcentagem de variáveis com >= 17.0% de dados faltantes: 77.47%


Total de campos com dados >= 18.0% faltantes- 141
Porcentagem de

### Pela observação, temos colunas com mais do que 99.5% de dados faltantes, desta forma faremos a remoção destas colunas antes do prosseguimento das análises

In [128]:
#Removendo colunas com valores acima de 99.5% de dados faltantes
df_market_droped = df_market.drop(list(df_aux.query(f'tx_miss >= {0.995}').colunas),axis=1)

### Observações gerais:
- Dados faltantes são um desafio no dataset
- Uma exploração maior deve ser feita sobre os dados faltantes

### 2. Estudos para entender o comportamento das variáveis com dados faltantes
1. Entender o comportamento dos dados faltantes com base na variável _de_ramo_ 
2. Entender o commportamento dos dados faltantes com base na variável _setor_

#### Criando dataframe com as linhas sendo as variáveis do dataframe origina e as colunas os ramos de atividades

In [129]:
#criando vetor com os ramos de atividades
de_ramo = df_market_droped.de_ramo.unique()

In [159]:
df_compare_missing = pd.DataFrame(data={'variaveis':df_market_droped.columns})

In [160]:
cont = 0
for ramo in de_ramo:
    cont +=1
    df_ramo = df_market_droped.query(f'de_ramo == "{ramo}"')
    df_aux = create_metadata(df_ramo)
    #df_aux.rename(columns={'tx_miss':ramo},inplace=True)
    df_compare_missing.insert(1,ramo,df_aux.tx_miss.values)
df_compare_missing

,variaveis,FARMACEUTICA,ORGANISMOS INTERNACIONAIS,INDUSTRIA AUTOMOTIVA,SERVICOS SOCIAIS,ATIVIDADES IMOBILIARIAS,SERVICOS DE SANEAMENTO BASICO,ELETROELETRONICOS,INDUSTRIAS EXTRATIVAS,ELETRICIDADE E GAS,...,INDUSTRIA DIGITAL,COMERCIO VAREJISTA,BENS DE CONSUMO,SERVICOS ADMINISTRATIVOS,QUIMICA-PETROQUIMICA,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVICOS DIVERSOS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVICOS DE ALOJAMENTO/ALIMENTACAO,INDUSTRIA DA CONSTRUCAO
0,Unnamed: 0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,id,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,fl_matriz,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,de_natureza_juridica,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,sg_uf,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,qt_funcionarios_24meses,0.77,0.78,0.60,0.79,0.68,0.66,0.61,0.64,0.76,...,0.73,0.76,0.77,0.71,0.63,0.80,0.93,0.76,0.76,0.77
165,tx_crescimento_12meses,0.91,0.89,0.69,0.83,0.80,0.74,0.69,0.73,0.80,...,0.82,0.83,0.83,0.79,0.71,0.86,0.95,0.83,0.83,0.85
166,tx_crescimento_24meses,0.86,0.89,0.69,0.82,0.79,0.74,0.68,0.73,0.79,...,0.82,0.83,0.83,0.79,0.71,0.86,0.95,0.83,0.83,0.85
167,tx_rotatividade,0.77,0.78,0.60,0.79,0.68,0.66,0.61,0.64,0.76,...,0.73,0.76,0.77,0.71,0.63,0.80,0.93,0.76,0.76,0.77


In [161]:
df_compare_missing.set_index('variaveis',inplace=True)

In [178]:
fig = go.Figure()
for col in df_compare_missing.columns:
    fig.add_trace(go.Scatter(x=df_compare_missing.index,y=df_compare_missing[col],mode='lines+markers'))
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed